In [1]:
import os
import sys

from dotenv import load_dotenv

sys.path.append(os.path.abspath(os.pardir))
load_dotenv('../.env')

True

In [3]:
from langchain_google_vertexai import VertexAI

llm = VertexAI(model_name='gemini-pro', temperature=0)

In [16]:
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.embeddings.voyageai import VoyageEmbeddings
from langchain.llms.huggingface_hub import HuggingFaceHub
from langchain.embeddings.huggingface_hub import HuggingFaceHubEmbeddings


embedding = VertexAIEmbeddings('textembedding-gecko@001')

# XXX: 性能がgeckoと比較して著しく悪い!!!
# embedding = GoogleGenerativeAIEmbeddings(model='models/embedding-001', google_api_key=os.environ['GOOGLE_GENERATIVE_LANGUAGE_API_KEY'])

# XXX: ERROR!!!embed_contentメソッドが実装されていないのでretrieverのembeddingとして使えない!!
# embedding = GoogleGenerativeAIEmbeddings(model='models/embedding-gecko-001', google_api_key=os.environ['GOOGLE_GENERATIVE_LANGUAGE_API_KEY'])

# XXX: responseがとても遅い!性能も悪い！!
# embedding = HuggingFaceHubEmbeddings(repo_id='intfloat/multilingual-e5-base')

# TODO: API KEYの有効化(課金)、性能比較
# embedding = VoyageEmbeddings(voyage_api_key=os.environ['CLAUDE_API_SECRET_KEY'])Ø

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.web_base import WebBaseLoader

loader = WebBaseLoader(web_path='https://python.langchain.com/docs/modules/memory/')
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [17]:
from langchain_community.vectorstores.chroma import Chroma

retriever = Chroma.from_documents(documents=splits, embedding=embedding) \
    .as_retriever(search_kwargs={'k': min(len(docs), 4)})

In [18]:
result = retriever('How to add memory to a chain? Please explain with an code example.')
result = retriever.get_relevant_documents('How to add memory to a chain? Please explain with an code example.')

In [19]:
print(result[0].page_content)

LangChain provides a lot of utilities for adding memory to a system.
These utilities can be used by themselves or incorporated seamlessly into a chain.Most of memory-related functionality in LangChain is marked as beta. This is for two reasons:Most functionality (with some exceptions, see below) are not production readyMost functionality (with some exceptions, see below) work with Legacy chains, not the newer LCEL syntax.The main exception to this is the ChatMessageHistory functionality. This functionality is largely production ready and does integrate with LCEL.LCEL Runnables: For an overview of how to use ChatMessageHistory with LCEL runnables, see these docsIntegrations: For an introduction to the various ChatMessageHistory integrations, see these docsIntroduction​A memory system needs to support two basic actions: reading and writing.
Recall that every chain defines some core execution logic that expects certain inputs.


In [20]:
result[0].metadata

{'description': 'Most LLM applications have a conversational interface. An essential component of a conversation is being able to refer to information introduced earlier in the conversation.',
 'language': 'en',
 'source': 'https://python.langchain.com/docs/modules/memory/',
 'title': '[Beta] Memory | 🦜️🔗 Langchain'}

In [22]:
docs[0].metadata['source'] == result[0].metadata['source']

True

性能はClaude以外では圧倒的にgeckoモデルが強い。

ただし、geckoモデルを使用しても、example codeは取得できていない。

結局、geminiを使う場合はある程度長いcontextでも問題なく入れられるので、「基本はWebサイト全体のpage contentをcontextにそのまま代入」「複数のWebサイトをRAGに使う場合のみretrieverを使用してmetadataから該当するWebサイトを特定してやはりサイト全体のpage contentを代入」の方が精度がいい可能性が高い。